In [21]:
# Recall the data from website
import requests
import pandas as pd

url = "https://hayday.fandom.com/wiki/Goods_List"
headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/124.0 Safari/537.36"
    )
}
html = requests.get(url, headers=headers, timeout=30).text  
df = pd.read_html(html, flavor="lxml")[0]

# Filtered with level
df["Level"] = (
    df["Level"]
    .astype(str)
    .str.extract(r"(\d+)", expand=False)    
    .astype(float)                          
    .astype("Int64")                        
)
filtered = df[df["Level"] <= 17]

/var/folders/97/ch83kwyd58z8krz3tltkbxkm0000gn/T/ipykernel_12171/2487487634.py:14: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(html, flavor="lxml")[0]


In [22]:
filtered

,Name,Level,Max. price,Time,XP,Needs,Source,Per boat crate
0,Wheat,1,3,2 min,1,Wheat (1),Field (1st crop),50-101
1,Egg,1,18,20 min,2,Chicken feed (1),Chicken (1st animal product),15-24
2,Corn,2,7,5 min,1,Corn (1),Field (2nd crop),50-87
3,Bread,2,21,5 min ★★★ 4 min,3,Wheat (3),Bakery,14-26
4,Chicken feed,3,7,5 min ★★★ 4 min,1,Corn (1) Wheat (2),Feed Mill,NaN
5,Soybean,5,10,20 min,2,Soybean (1),Field (3rd crop),24-48
6,Cow feed,6,14,10 min ★★★ 8 min,2,Corn (1) Soybean (2),Feed Mill,NaN
7,Milk,6,32,1 h,3,Cow feed (1),Cow (2nd animal product),8-16
8,Cream,6,50,20 min ★★★ 17 min,6,Milk (1),Dairy,8-12
9,Sugarcane,7,14,30 min,3,Sugarcane (1),Field (4th crop),14-31


In [3]:
df.dtypes


Name              object
Level             object
Max. price         int64
Time              object
XP                object
Needs             object
Source            object
Per boat crate    object
dtype: object

In [4]:
df.loc[89]


Name                           Wheat bundle
Level                           34 (Rep. 3)
Max. price                               50
Time              1 h 30 min ★★★ 1 h 16 min
XP                                       10
Needs                            Wheat (75)
Source                            Feed Mill
Per boat crate                          NaN
Name: 89, dtype: object

In [5]:

# there is another culprit who has funny Time `Instantor 20 h`
df.loc[64]
     

Name                                                    Fish fillet
Level                                                            27
Max. price                                                       54
Time                                                Instant or 20 h
XP                                                             4-20
Needs             Lure (1) or Fishing net (1) Fishing spot (any ...
Source                                    Fish (5th animal product)
Per boat crate                                                  6-9
Name: 64, dtype: object

In [8]:
df["Level"]

0        1
1        1
2        2
3        2
4        3
      ... 
396    121
397    122
398    124
399    125
400    126
Name: Level, Length: 399, dtype: object

In [9]:
# remove the culprits
df = df.drop(df.index[[89]])
df = df.drop(df.index[[64]])

df['Level'] = df['Level'].astype(int)

df.dtypes

ValueError: invalid literal for int() with base 10: '34 (Rep. 3)'

In [ ]:
# Here I convert the `1 d 20 h 30 min` like time to actual minutes
# So I can do the time versus price analysis later

# This function work with each row
def format_time(row):
  time_string = ""
  time_in_minutes = 0

  if '★' in row['Time']:
    time_string = row['Time'].split("★")[0]
  else:
    time_string = row['Time']

  for i in time_string:
    if i == 'd':
      time_in_minutes += int(time_string.split('d')[0].strip()) * 24 * 60
      time_string = time_string[time_string.find('d')+1:]
    if i == 'h':
      time_in_minutes += int(time_string.split('h')[0].strip()) * 60
      time_string = time_string[time_string.find('h')+1:]
    if i == 'm':
      time_in_minutes += int(time_string.split('m')[0].strip())
  
  return time_in_minutes

df['Time'] = df.apply(format_time, axis=1)

df

In [10]:
'''
This is where I made a new column of `Price/min` which gives us a clear 
picture of which products are better to farm, has more money value per min
'''
df['Price/min'] = df.apply(lambda x: round((x['Max. price'] / x['Time']), 2) if x['Time'] != 0 else 0, axis=1)


##### CHANGE OUR LEVEL HERE #####
level = 50
#####


# now sort using the `Price/min`
df[df['Level'] <= level].sort_values(['Price/min'], ascending=[False])

TypeError: unsupported operand type(s) for /: 'int' and 'str'

In [11]:
# We can also find most profitable item for a source, like the below one

df[(df['Level'] <= level) & (df['Source'] == 'Dairy')].sort_values(['Price/min'], ascending=[False])
     


NameError: name 'level' is not defined

In [12]:
# Best items from each source

idx = df.groupby('Source')['Price/min'].transform(max) == df['Price/min']

df[idx][df['Level'] <= level].sort_values(['Price/min'], ascending=[False])
     

KeyError: 'Column not found: Price/min'

In [ ]:
'''
OK now the funny thing is we have to wait for the items we need to make the product.
So here are some suggestions and analysis -

# If we produce the items we need parallelly, 
  the most time will be consumed by the product which is produced by machines.
  ( We can plant several crops right? ;) )
  Like - We need 3 coffee beans and 1 white sugar to make Espresso which is made in only 5 min!
  But we need 1 white sugar which takes 40 mins to produce in sugar mill. 
  And we can plant as many coffee bush we want and get coffee beans in 25 hours :3
  So practically we are not making money fast with Espresso.

Now we will try to make a matrix by which we can calculate price/min with dependant items too.
We can name it `Price/min with needs`
'''
import inflect
p = inflect.engine()


list_of_parallel_sources = ['Field', 'Cow', 'Chicken', 'bush', 'Pig', 'Sheep', 'tree']

def process_name(name):
  name = name[:-2].strip()
  if p.singular_noun(name) is False:
    return name
  return p.singular_noun(name)

def match_two_lists(listA, listB):
  for i in listA:
    if i in listB:
      return True
  return False


def calc_price_per_min_with_needs(row):
  if pd.isnull(row['Needs']):
    return pd.Series((row['Time'], round((row['Max. price'] / row['Time']), 2) if row['Time'] != 0 else 0))
  needs = row['Needs'].split(")")
  total_time = 0
  for need in needs:
    if need == '':
      continue

    match_item = df.loc[df['Name'] == process_name(need)]
    try:
      if match_two_lists(match_item['Source'].item().split(" "), list_of_parallel_sources):
        total_time += match_item['Time'].item()
      else:
        total_time += match_item['Time'].item() * int(need[-1])
    except Exception as e:
      print(need)
      print(match_item)
      # print(e)

  if total_time < row['Time']:
    return pd.Series((row['Time'], round((row['Max. price'] / row['Time']), 2) if row['Time'] != 0 else 0))
  else:
    return pd.Series((total_time, round((row['Max. price'] / total_time), 2) if row['Time'] != 0 else 0))


df[['Actual time with needs', 'Price/min with needs']] = df.apply(calc_price_per_min_with_needs, axis=1)

df[df['Level'] <= level].sort_values(['Price/min with needs'], ascending=[False])


In [13]:
df[(df['Level'] <= level) & (df['Source'] == 'Pie Oven')].sort_values(['Price/min with needs'], ascending=[False])


NameError: name 'level' is not defined

In [ ]:
# Best items from each source

idx = df.groupby('Source')['Price/min with needs'].transform(max) == df['Price/min with needs']

distinct_source_df = df[idx]

distinct_source_df[distinct_source_df['Level'] <= level].sort_values(['Price/min with needs'], ascending=[False])
     
